По моему мнению, данные для таблицы взяты с сайта www.metacritic.com, так как путем проверки найдены совпадения. Проблема в том, что сайт блокирует попытки парсинга его данных, поэтому я обратился к сайтам, аффилированным с ним, точнее к www.gamespot.com.
Они оба принадлежат одной группе.  
КПД парсера составило около 20%, так как сайт форматирует названия игр из таблицы по своему усмотрению и нужна кропотливая ручная работа по составлению ссылок.   
Из 324 удачных запросов по играм, у которых нет пропусков, обнаружено 228 совпадения, а максимальное отличие не превышает 5%.
По пользовательским оценкам данные различаются сильнее, так как аудитории разные. Среднее оклонение рейтинга получилось 0.8
Сама работа парсера медленная, чтобы запросить 7312 игр нужно три часа.

Также прилагается парсер сайта ESRB.com для пользовательских рейтингов

In [ ]:
#Чтобы снизить вероятность того, что клиентский сервер примет вас за бота

In [2]:
pip install fake-useragent

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

In [ ]:
#Приведение колонок с текстовой информацией к нижнему регистру
text_cols = ['name', 'platform', 'genre','rating']
for col in text_cols:
    data[col] = data[col].str.lower()
    
# Создание списка игр, информацию о которых нужно уточнить
games_to_find = data[data.isnull().any(axis=1)]['name'].unique() 

domain = r'https://www.gamespot.com/games'   #домен сайта
formatted_names = [] #Список отформатированных имен, которые принимает сайт в адресной строке браузера
list_of_links = []   #список ссылок на игры

#Преобразование имен из таблицы в формат, принимаемый сайтом
for name in games_to_find:
    name = str(name) #конвертировать в тип строки, чтобы превратить точку у числе типа float в конце просто в точку
    name = name.strip(' ')
    #замена символа & на and
    name = re.sub('[&]+', 'and', str(name))
    #замена любых символов, кроме букв и цифр на дефисы
    name = re.sub('[^A-Za-z0-9]+', '-', str(name))
    name = re.sub('[-, /]*$','',str(name))
    formatted_names.append(name)
    link = domain+'/'+name    #построение ссылки
    list_of_links.append(link) #добавление ссылки в список

#Словарь ссылок и названий    
names_links_pairs = dict(zip(list_of_links, games_to_find))

##--------------------ТЕЛО ПАРСЕРА------------------------------
exception_list = [] # Список, куда будут заноситься названия, ссылки и ошибки 
game_info = {}   #Словарь, который будет создан парсером с нужной информацией
main_list = []   #Список словарей game_info

for link in list_of_links:
    game_info = {}
    try:
        # значение header вашего браузера меняется на случайное из списка, чтобы труднее сойти за бота
        ua = UserAgent()
        header = {'User-Agent':str(ua.chrome)}         
        htmlContent = requests.get(link, headers=header)
        soup = BeautifulSoup(htmlContent.text)
        
        #Название игры
        title = soup.find('h1',{'class':'gameObject__title'})  
        title = re.sub('\n','',str(title.text))
        game_info['name'] = title .lower()   
        
        #Год выпуска
        release_date = soup.find('ul', {'class':"kubrick-info__releasedate"}).find('li').find('span').text
        # на сайте дата выхода игры указывается в полном формате, где 4 цифры года идут в конце,
        #поэтому нужно найти 4 цифры, идущие подряд
        year = int(re.search('[0-9]{4}',release_date).group())
        game_info['year_of_release'] =  year        
        
        #Платформа
        ultag = soup.find('div', {'class':"gameObject__description"}).find('ul',{'class':'system-list'})
        platforms = [] #список платформ
        #Непосредственно текст  содержит тег <li>
        for li in ultag.find_all('li'):
            platforms.append(li.text.lower()) #платформ может быть несколько, поэтому добавляем каждую в список
        game_info['platform'] = platforms[0]  #главной будет первая        
         
        #Оценки
        critic_score= soup.find('div',{'class':'reviewObject'}).find('dl',{'class':'reviewObject__metacritic'}).find('dd').text
        if critic_score =='--':
            game_info['critic_score'] = np.nan
        else:
            game_info['critic_score'] = int(critic_score)
        user_score = soup.find('div',{'class':'reviewObject'}).find('dl',{'class':'reviewObject__userAvg'}).find('a').text
        
        game_info['user_score'] = float(user_score)
        #main_list.append(game_info)  
    except BaseException as e:
        exception_list.append(" Filename: {}, Link: {}, Error : {}".format(names_links_pairs[link],link, str(e)))
    main_list.append(game_info)
#---------------------------------------------КОНЕЦ ПАРСЕРА-------------------------------------------    
#print(main_list)
#print(exception_list)

In [ ]:
save_folder = r'Адрес к папке с файлами'

In [ ]:
#Файл с новой информацией к играм с пропусками
try:
    gamestop_data_2 = pd.read_csv(save_folder+r'\gamestop_data_2.csv')
except:
    print("Укажите путь к файлу")

#Файл с проверочной информацией по играм, у которых нет пропуски, 
#но надо сравнить насколько отличаются оценки здесь и там на сайте
try:
    gamestop_data_val = pd.read_csv(save_folder+r'\gamestop_data_val.csv')
except:
    print("Укажите путь к файлу")

#Файл для ленивых с объединенной информацией из предыдущих двух по полям name, platform.
## Колонки, оканчивающиеся на _gs - данные с сайта по играм для таблицы gamestop_data_2
##Колонки, оканчивающиеся на _val - данные с сайта по играм для таблицы gamestop_data_val
try:
    data_updated_full = pd.read_csv(save_folder+r'\data_updated_full.csv')
except:
    print("Укажите путь к файлу")

## Парсер сайта ESRB.com

In [ ]:
#Данные
rating_off = data[data['rating'].isna()].reset_index()
ratingDict_off = rating_off[['name','platform','year_of_release']].to_dict(orient = 'index')

#Cловарь кодировок названий платформ согласно сайту, для тех кто не вошел сюда стоит значение Other
platform_encoding = {'wiiu':'Wii U',
                      'x360':'Xbox 360',
                      'ps3':'PlayStation 3',
                      'ps4':'PlayStation 4',
                      'pc':'PC',
                      '3ds':'Nintendo 3DS',
                      'xone':'Xbox One'    
}
for name in data['platform'].unique():
    if name not in platform_encoding.keys():
        platform_encoding[name] = 'Other'
        #platform_encoding[name] = 'All Platforms'
        
## Форматирование части ссылки, относящейся к платформе
for key in platform_encoding:
    platform_encoding[key] = re.sub(' ','%20', str(platform_encoding[key])) 
print(platform_encoding )

Построитель ссылок:

domain = 'https://www.esrb.org/search/?searchKeyword='
rating_part = '&rating=E%2CE10%2B%2CT%2CM%2CAO&descriptor=All%20Content&'
#page = 'pg=1'
ending = '&searchType=All&ielement[]=all'
pl_part = '&platform='
formatted_names = []
link_list_test = {}
for key, dict_ in ratingDict_off.items():
    d_temp = {}
    name_form = str(dict_['name']).strip(' ')#str() нужно, чтобы символы вроде точек у цифр типа float etc были строковым типом 
    name_form = re.sub(' ', '%20', name_form)
    if  '\'' in dict_['name']: # проверка на наличие апострофа в слове
        name_form = re.sub("\'", '%27', name_form)
    if ":" in dict_['name']:  # проверка на наличие двоеточия
        name_form = re.sub(':', '%3A', name_form)
    if "!" in dict_['name']: #проверка на наличие восклицательного знака
        name_form = re.sub('!', '%21', name_form)    
    if re.match("pokemon\D+/pokemon\D+",dict_['name'] ): # in case of pokemon red/pokemon blue pattern
        name_form = re.split("/",name_form)[0] 
        #построение ссылки
    link = domain + name_form+pl_part+platform_encoding[dict_['platform']]+rating_part
    formatted_names.append(name_form)    
    ratingDict_off[key]['link'] = link  #добавление ссылки в главный словарь под ключом "link"    
#print(ratingDict_off)

Парсер

domain = 'https://www.esrb.org/search/?searchKeyword='
rating_part = '&rating=E%2CE10%2B%2CT%2CM%2CAO&descriptor=All%20Content&'
#page = 'pg=1'
ending = '&searchType=All&ielement[]=all'
pl_part = '&platform='
main_list_off = {}
exception_list_off = []
for key, dict_ in  ratingDict_off.items():       
    try:
        
        ua = UserAgent()
        header = {'User-Agent':str(ua.chrome)}
        for page in range(1,4):
            htmlContent = requests.get(dict_['link']+"pg={}".format(page)+ending, headers=header)
            #print(dict_['link']+"pg={}".format(page)+ending)
            soup = BeautifulSoup(htmlContent.text)
            found_titles = []
            for tag in soup.find_all('div', {'id':'results'}):                
                for tag in tag.find_all('div',{'class':'heading'}):
                    title = tag.find('h2').text
                    #temp = []
                    #temp.append(title)
                    #found_titles[key] = temp
                    found_titles.append(title)            
                ## Cортировка
                    sorted_title = sorted(title.strip().lower())
                #title_to_compare = ratingDict_on[0]['name']
                    if re.match("pokemon\D+/pokemon\D+",dict_['name']):
                            title_to_compare = re.split("/",dict_['name'])[0]
                            #print('if pokemon case compare to:', title_to_compare)
                    else:
                            title_to_compare = dict_['name']
                            #print('if not a pokemon case compare to: ', dict_['name'])
                    sorted_title_compareTo = sorted(title_to_compare.strip().lower())
                #Если найдено совпадение
                    if sorted_title == sorted_title_compareTo:
                            #print('title:', title,' compare to: ',title_to_compare )
                            platform = tag.find('div',{'class':'platforms'})
                            esrb = tag.parent.find('div',{'class':'content'}).find('img')
                        #print(esrb.text)
                            gathered_data = {}
                            gathered_data['name_on_site'] = title.lower()
                            gathered_data['platforms'] = platform.text.lower()
                            gathered_data['rating'] = esrb['alt'].lower()
                            #gathered_data['link'] = dict_['link']+"pg={}".format(page)+ending
                            #print(gathered_data)
                    else:
                            continue
        #print (found_titles)
    except  BaseException as e:
        exception_list_off.append(" Filename: {}, Link: {}, Error : {}".format(dict_['name'], dict_['link'], str(e)))    
    ratingDict_off[key]['gathered_info'] = gathered_data
    if gathered_data['name_on_site'] not in main_list_off.values():
        main_list_off[key] = gathered_data
#print(main_list_test)    
       

esrb_new = pd.DataFrame(main_list_off)
parced_ESRB_full = pd.DataFrame(ratingDict_off)
display(parced_ESRB_full.T.head())
display(esrb_new.T.head())

In [ ]:
## КОД ДЛЯ ПОЛУЧЕНИЯ НЕБОЛЬШОЙ СТАТИСТИКИ
#data_updated_full = data_updated_full.drop_duplicates() ## в новых данных есть дубликаты
#data_temp = data_updated_full[~data_updated_full.isnull().any(axis = 1)].query('user_score !="tbd"')
#data_temp ['user_score'] = data_temp ['user_score'].astype('float')
#data_temp['diff_cs_pct'] = ((data_temp['critic_score'] - data_temp['critic_score_val'])/data_temp['critic_score_val']*100)
#data_temp['diff_us_pct'] = (data_temp['user_score'] - data_temp['user_score_val'])/data_temp['user_score_val']*100
#display(data_temp[['name','critic_score','critic_score_val','diff_cs_pct','user_score','user_score_val','diff_us_pct']])

#Количество совпадений
#print('Количество совпадений', data_temp[data_temp['critic_score']==data_temp['critic_score_val']]['name'].count())
#display(data_temp['diff_cs_pct'].describe())
#print(('Среднее отклонение значений user_score', data_temp['user_score'] - data_temp['user_score_val']).abs().mean())